In [ ]:
import pandas as pd  #იმპორტები
import numpy as np
import torch
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
!pip install evaluate
!pip install -q transformers datasets evaluate gradio accelerate scikit-learn matplotlib
import evaluate
import gradio as gr

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset = load_dataset("Arseniy-Sandalov/Georgian-Sentiment-Analysis")
print(dataset)

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # ვწმენდ დატას და ვშლი ლინკებს
    text = re.sub(r'\s+', ' ', text).strip() # ზედმეტი სფეისების წაშლა
    return text

dataset = dataset.map(lambda x: {"text": clean_text(x["text"])})

In [ ]:
dataset['train'][0] #შევამოწმე მუშაობა

In [ ]:
dataset = dataset.filter(lambda x: x["sentiment"] in ["Negative", "Neutral", "Positive"])

In [ ]:
dataset_no_mixed = dataset.filter(   ## ამოვშალე mixed ტექსტი
    lambda x: x["sentiment"] in ["Negative", "Neutral", "Positive"]
)

In [ ]:
from collections import Counter  #შევამოწმე

labels = [ex["sentiment"] for ex in dataset_no_mixed['train']]
Counter(labels)

In [ ]:
def filter_labels(example): # მივანიჭე Labels
    return example['sentiment'] in ["Negative", "Neutral", "Positive"]

dataset['train'] = dataset['train'].filter(filter_labels)

In [ ]:
label_mapping = {"Negative":0, "Neutral":1, "Positive":2}
def encode_labels(example):
    example["label"] = label_mapping[example["sentiment"]]
    return example

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
split_data = dataset["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = split_data["train"]
test_dataset = split_data["test"]

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Arseniy-Sandalov/Georgian-Sentiment-Analysis")

print(dataset['train'].column_names)

In [ ]:
from datasets import load_dataset
import pandas as pd

raw_dataset = load_dataset("Arseniy-Sandalov/Georgian-Sentiment-Analysis")

df = pd.DataFrame(raw_dataset['train'])

df = df.rename(columns={'sentiment': 'label'})
df = df[['text', 'label']]

from datasets import Dataset
full_dataset = Dataset.from_pandas(df)

dataset_split = full_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
test_dataset = dataset_split["test"]

print("დატასეტი მზადაა", train_dataset.column_names)

In [ ]:
label_map = {"negative": 0, "neutral": 1, "positive": 2}

def map_labels(example):
    if isinstance(example["label"], str):
        return {"label": label_map.get(example["label"].lower(), 1)}
    return {"label": int(example["label"])}

train_dataset = train_dataset.map(map_labels)
test_dataset = test_dataset.map(map_labels)

print("ლეიბლები გადაკონვერტირდა ციფრებში")

In [ ]:
def encode_labels(example):
    val = example["label"]

    if isinstance(val, int):
        return {"label": val}

    mapping = {"negative": 0, "neutral": 1, "positive": 2}

    if str(val).isdigit():
        return {"label": int(val)}

    return {"label": mapping.get(str(val).lower(), 1)}

train_dataset = train_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

print(type(train_dataset[0]['label']))

In [ ]:
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

labels = train_dataset["label"]

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

weights_tensor = torch.tensor(class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")

print("weights_tensor შეიქმნა")
print("წონებია:", weights_tensor)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# ვითვლით წონებს
labels = train_dataset["label"]
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()

In [ ]:

from collections import Counter
print("არსებული ლეიბლები:", Counter(dataset_split['train']['label']))

label_map = {"negative": 0, "neutral": 1, "positive": 2}

def map_labels(example):
    val = str(example["label"]).lower()
    return {"label": label_map.get(val, 1)}

train_ds = dataset_split['train'].map(map_labels)

negatives = train_ds.filter(lambda x: x['label'] == 0)
neutrals  = train_ds.filter(lambda x: x['label'] == 1)
positives = train_ds.filter(lambda x: x['label'] == 2)

print(f"ნეგატიური: {len(negatives)}, ნეიტრალური: {len(neutrals)}, პოზიტიური: {len(positives)}")

In [ ]:
import gc
from accelerate import Accelerator

gc.collect()
torch.cuda.empty_cache()
Accelerator().free_memory()

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3)
model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import evaluate

# ჩავტვირთე accuracy-ს მეტრიკა
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("final_model")
tokenizer.save_pretrained("final_model")

In [ ]:
from tqdm.auto import tqdm

tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
# ვამოწმებ შეცდომებს რომ accuracy ავამაღლო
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

wrong_preds = []
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}

print("შეცდომების ძებნა...")

for i in tqdm(range(len(tokenized_test))):

    inputs = {
        "input_ids": tokenized_test[i]["input_ids"].unsqueeze(0).to(device),
        "attention_mask": tokenized_test[i]["attention_mask"].unsqueeze(0).to(device)
    }

    true_label = tokenized_test[i]["label"].item() # .item() სჭირდება, რომ რიცხვი ამოვიღოთ

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    pred_label = torch.argmax(logits, dim=-1).item()

    if pred_label != true_label:
        text = tokenizer.decode(tokenized_test[i]["input_ids"], skip_special_tokens=True)

        wrong_preds.append({
            "Text": text,
            "True Label": id2label[true_label],
            "Predicted Label": id2label[pred_label]
        })

df_errors = pd.DataFrame(wrong_preds)

print(f"\nსულ ნაპოვნია {len(df_errors)} შეცდომა.")
print("-" * 50)
if len(df_errors) > 0:
    display(df_errors.head(10))
else:
    print("არცერთი შეცდომა არ უპოვია)")

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask).logits

prediction = torch.argmax(logits, dim=1).item()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

all_preds = []
all_labels = []

model.eval()
for i in range(len(tokenized_test)):
    input_ids = tokenized_test[i]["input_ids"].unsqueeze(0).to(device)
    attention_mask = tokenized_test[i]["attention_mask"].unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits

    all_preds.append(torch.argmax(logits, dim=1).item())
    all_labels.append(tokenized_test[i]["label"].item())

# მატრიცის აგება
cm = confusion_matrix(all_labels, all_preds)
labels = ["Negative", "Neutral", "Positive"]

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('მოდელის ნათქვამი (Predicted)')
plt.ylabel('სინამდვილე (True)')
plt.title('Confusion Matrix - სად ეშლება მოდელს?')
plt.show()

In [ ]:
model_path = "./final_model"  #რეპოზიტორის სახელი
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

labels = ["Negative", "Neutral", "Positive"]

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    return {labels[i]: float(probabilities[0][i]) for i in range(3)}

demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(placeholder="ჩაწერე კომენტარი აქ...", lines=2),
    outputs=gr.Label(num_top_classes=3),
    title="ქართული სოციალური მედიის სენტიმენტ-ანალიზი",
    description="შეიყვანეთ ტექსტი, რათა გაიგოთ, არის ის პოზიტიური, ნეგატიური თუ ნეიტრალური."
)

demo.launch()

In [ ]:
import torch
from tqdm.auto import tqdm

# ვიგებ accuracy-ს
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

correct = 0
total = len(tokenized_test)

print("სიზუსტის დათვლა")

for i in tqdm(range(total)):

    inputs = {
        "input_ids": tokenized_test[i]["input_ids"].unsqueeze(0).to(device),
        "attention_mask": tokenized_test[i]["attention_mask"].unsqueeze(0).to(device)
    }
    true_label = tokenized_test[i]["label"].item()

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.argmax(outputs.logits, dim=-1).item()

    if prediction == true_label:
        correct += 1

#შედეგი გამომაქვს პროცენტებში
accuracy = correct / total
print("\n" + "="*30)
print(f"საბოლოო სიზუსტე (Accuracy): {accuracy:.2%}")
print("="*30)